In [ ]:
import uproot as ur
import matplotlib.pyplot as plt
import numpy as np
import awkward as ak
from scipy.stats import norm
from scipy.optimize import curve_fit

## Get data, transform ROOT tree into array

In [ ]:
# Parameters 
energy = 1 # Energy in GeV. Available energies are 1, 2, 5, 10, 20, 50 GeV.
particle = 'pi+' # Particle incident on detectors


files = [] # List to contain all the data file names

# Loop through some large number to obtain all the available files so we don't have to look them up manually. 
for file_number in range(1, 101):
    try:
        file_name = 'root://sci-xrootd.jlab.org//osgpool/eic/ATHENA/RECO/SINGLE/{particle}\
/{energy}GeV/3to50deg/{particle}_{energy}GeV_3to50deg.{file_number:04d}.root:events'.format(energy = energy, particle = particle, file_number = file_number)
        ur.open(file_name) # Open each file. If the file doesn't exist, it will be skipped and not be added to files list.
        files.append(file_name)
    except:
        continue
        
print("Number of files: {NumFiles}.".format(NumFiles = len(files)))

# Combining all the data files that we found
# Cannot combine each desired branch into one big array. Have to either combine all branches (including ones we don't want like Barrel branches) or make separate arrays for each branch we want.
EcalP_events = ur.concatenate(files, ["EcalEndcapPClusters.edep", "EcalEndcapPClusters.polar.theta"], library = 'np')
HcalHadron_events = ur.concatenate(files, ["HcalHadronEndcapClusters.edep","HcalHadronEndcapClusters.polar.theta"], library = 'np')


In [ ]:
def Angle_Energy(detector_events = HcalHadron_events, detector_name = "HcalHadronEndcapClusters", desired_angle = 3):
    num_events = len(detector_events["{detector}.polar.theta".format(detector = detector_name)])
    energies = []
    for i in range(num_events):
        num_hits = len((detector_events["{detector}.polar.theta".format(detector = detector_name)][i]))
        hit_energy = []
        for j in range(num_hits):
            angle = (detector_events["{detector}.polar.theta".format(detector = detector_name)][i][j])
            if angle >= desired_angle - 1 or angle <= desired_angle + 1:
                hit_energy.append(detector_events["{detector}.edep".format(detector = detector_name)][i][j])
        energies.append(hit_energy)
    return energies

In [ ]:
# Convert angles to degrees
HcalHadron_events["HcalHadronEndcapClusters.polar.theta"] *= 180/np.pi
EcalP_events["EcalEndcapPClusters.polar.theta"] *= 180/np.pi

In [ ]:
# If you want a specific angle range, set filter_angle to True and input angle into angle variable. Will select events with angles "angle" +- 1 degree 
filter_angle = False
angle = 3 # Angle in degrees

Hcal_energies = []
Ecal_energies = []
if filter_angle:
    Hcal_energies = Angle_Energy(HcalHadron_events, "HcalHadronEndcapClusters", angle)
    Ecal_energies = Angle_Energy(EcalP_events, "EcalEndcapPClusters", angle)
else:
    Hcal_energies = HcalHadron_events["HcalHadronEndcapClusters.edep"]
    Ecal_energies = EcalP_events["EcalEndcapPClusters.edep"]

In [ ]:
# Weighting the hadron endcap energies
w0 = 0.1
weights = np.arange(w0 + 0.1, 2.2, 0.1)

EW = [] # EW will hold the weighted total energies for each weight in "weights"
for i in range(len(weights)):
    EW.append([])

num_events = len(Hcal_energies)

for i in range(num_events):
    event_edep = np.sum(Hcal_energies[i])
    for j in range(len(weights)):
        EW[j].append(event_edep/weights[j])

In [ ]:
def gaussian(x, a, mean, sigma):
    return a * np.exp(-((x - mean)**2 / (2 * sigma**2)))


hist_data, bins, patches = plt.hist(EW[1], 1000, (0.5, energy*100+.5), density=True,facecolor='g', alpha=0.75)

binscenters = np.array([0.5 * (bins[i] + bins[i+1]) for i in range(len(bins)-1)])

mu, sigma = norm.fit(EW[1])
print(mu, sigma)
print(mu/sigma)
fit_curve = norm.pdf(bins, mu, sigma)
#popt, pcov = curve_fit(gaussian, xdata=binscenters, ydata=hist_data, p0=[4000, mu, sigma],maxfev=50000)

plt.plot(bins, fit_curve)

### Next steps:
### ~1) Filter the angles. Choose a certain angle, then fill arrays of edep for that angle~
### 2) Fill a weighting histogram and fit it with a Gaussian to obtain the optimal weight
### 3) Fill a new histogram with the optimally weighted energy dependence and fit it to obtain the energy resolution
### 4) Do this for each energy and then plot (like EIC_EndCap.C)
